In [1]:
def is_float(element) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

def is_int(element) -> bool:
    try:
        int(element)
        return True
    except ValueError:
        return False

In [2]:
from sys import argv

fake_argv = 'update_ohlcv_and_forecast.py  --db_host=localhost --db_port=27017 --db_name=rata --db_col=rates_5m --symbol=EURUSD --interval=5 --kind=forex'
fake_argv = fake_argv.split()

#argv = fake_argv ####

_conf = dict()
for i in argv[1:]:
    if '=' in i:
        param = i.split('=')
        _conf[param[0].replace('--', '')] = param[1]

for i in _conf:
    b = _conf[i]
    if   b == 'True':
        _conf[i] = True
    elif b == 'False':
        _conf[i] = False
    elif is_int(b):
        _conf[i] = int(b)
    elif is_float(b):
        _conf[i] = float(b)
_conf

{'db_host': 'localhost',
 'db_port': 27017,
 'db_name': 'rata',
 'db_col': 'rates_5m',
 'symbol': 'EURUSD',
 'interval': 5,
 'kind': 'forex'}

In [3]:
import pandas as pd
import datetime as dt
from rata.marketon import get_data
from pymongo import MongoClient

client = MongoClient(_conf['db_host'], _conf['db_port'])
db = client[_conf['db_name']]
collection = db[_conf['db_col']]

In [4]:
#if not _conf['db_col'] in db.list_collection_names():

mydoc = collection.find({'symbol': _conf['symbol']}).sort('tstamp', 1).limit(9)
df = pd.DataFrame(mydoc)

if len(df) == 0:
    hours_back = 80 * _conf['interval']
else:
    df = df.groupby(['close', 'high', 'interval', 'low', 'open', 'status', 'symbol', 'tstamp', 'unix_tstamp', 'volume']).max('query_tstamp')
    df = df.reset_index()[['tstamp', 'interval', 'symbol', 'open', 'high', 'low', 'close', 'volume']].sort_values('tstamp')
    t1 = df['tstamp'].iloc[-1]
    t2 = dt.datetime.utcnow()
    t3 = t2 - t1
    hours_back = (t3.seconds // 3600) + 1

print(hours_back)

df = get_data.get_finnhub(symbol=_conf['symbol'], interval=_conf['interval'], kind=_conf['kind'], hours=hours_back)
df.index = df.index.to_series().apply(dt.datetime.isoformat)
df.reset_index(inplace=True)
df_dict = df.to_dict(orient='records')

400
https://finnhub.io/api/v1/forex/candle?symbol=OANDA:EUR_USD&resolution=5&from=1636866496&to=1638306496&token=c6emgsqad3ie37m1crq0
data_status, ok
min_rates, 5
min_now, 8


In [5]:
for r in df_dict:
    #collection.update_one(r, {'$set': r}, upsert=True)
    collection.insert_one(r, {'$set': r})

In [6]:
    mydoc = collection.find({'symbol': 'EURUSD'}).sort('tstamp', 1).skip(collection.count_documents({}) - 2100)
    df = pd.DataFrame(mydoc)
    df = df.groupby(['interval',  'status', 'symbol', 'tstamp', 'unix_tstamp', ]).mean()
    df = df.reset_index()[['tstamp', 'interval', 'symbol', 'open', 'high', 'low', 'close', 'volume']].sort_values('tstamp')
    df

,tstamp,interval,symbol,open,high,low,close,volume
0,2021-11-19 11:35:00,5,EURUSD,1.12961,1.12984,1.12948,1.12972,428.0
1,2021-11-19 11:40:00,5,EURUSD,1.12971,1.12981,1.12919,1.12953,494.0
2,2021-11-19 11:45:00,5,EURUSD,1.12951,1.12959,1.12901,1.12902,491.0
3,2021-11-19 11:50:00,5,EURUSD,1.12905,1.12922,1.12884,1.12890,418.0
4,2021-11-19 11:55:00,5,EURUSD,1.12889,1.12939,1.12850,1.12892,766.0
...,...,...,...,...,...,...,...,...
2095,2021-11-30 20:45:00,5,EURUSD,1.13319,1.13342,1.13314,1.13338,182.0
2096,2021-11-30 20:50:00,5,EURUSD,1.13340,1.13343,1.13314,1.13333,222.0
2097,2021-11-30 20:55:00,5,EURUSD,1.13334,1.13415,1.13332,1.13408,601.0
2098,2021-11-30 21:00:00,5,EURUSD,1.13410,1.13447,1.13396,1.13441,320.0


In [7]:
# predict here!

df_dict

[{'query_tstamp': '2021-11-30T18:08:28.111010',
  'tstamp': Timestamp('2021-11-14 22:00:00'),
  'unix_tstamp': 1636927200,
  'symbol': 'EURUSD',
  'interval': '5',
  'open': 1.14456,
  'high': 1.1446,
  'low': 1.14416,
  'close': 1.1445,
  'volume': 12,
  'status': 'ok',
  '_id': ObjectId('61a692ccbb8e9586aae26504')},
 {'query_tstamp': '2021-11-30T18:08:28.111010',
  'tstamp': Timestamp('2021-11-14 22:05:00'),
  'unix_tstamp': 1636927500,
  'symbol': 'EURUSD',
  'interval': '5',
  'open': 1.14447,
  'high': 1.14454,
  'low': 1.14421,
  'close': 1.14433,
  'volume': 23,
  'status': 'ok',
  '_id': ObjectId('61a692ccbb8e9586aae26505')},
 {'query_tstamp': '2021-11-30T18:08:28.111010',
  'tstamp': Timestamp('2021-11-14 22:10:00'),
  'unix_tstamp': 1636927800,
  'symbol': 'EURUSD',
  'interval': '5',
  'open': 1.14424,
  'high': 1.14435,
  'low': 1.14421,
  'close': 1.14435,
  'volume': 26,
  'status': 'ok',
  '_id': ObjectId('61a692ccbb8e9586aae26506')},
 {'query_tstamp': '2021-11-30T18:08: